In [2]:
import pandas as pd

In [3]:

#Dummy dataframe for coding FP Growth
df = pd.DataFrame()
df['id'] = [100, 200, 300, 400, 500]
df['itemls'] = [['f', 'a', 'c', 'd', 'g', 'i', 'm', 'p'], ['a', 'b', 'c', 'f', 'l', 'm', 'o'],['b', 'f', 'h', 'j', 'o'], \
               ['b', 'c', 'k', 's', 'p'],['a', 'f', 'c', 'e', 'l', 'p', 'm', 'n']]
data = {100:['f', 'a', 'c', 'd', 'g', 'i', 'm', 'p'], 200:['a', 'b', 'c', 'f', 'l', 'm', 'o'], 300:['b', 'f', 'h', 'j', 'o'], \
        400:['b', 'c', 'k', 's', 'p'], 500:['a', 'f', 'c', 'e', 'l', 'p', 'm', 'n']}

In [4]:
from collections import deque 

def traversetree(FPGrowth_obj):
    root = FPGrowth_obj.root 
    queue = deque([(root, root, 0)])
    while queue:
        parent_node, node, level = queue.popleft()
        print(f"{level = }")
        print(f"Parent: {parent_node.item}, Parent count: {parent_node.count}, Data: {node.item}, Count: {node.count}")
        for node_name in node.children:
            queue.append((node, node.children[node_name], level + 1))

def traverseheader(header_table):
    for key in header_table.keys():
        node = header_table[key]
        while node is not None:
            print(f"Header item: {key}, Link data: {node.item}, Link count: {node.count}")
            node = node.link 

In [16]:
#Global variable
id = 0
class Node:
    def __init__(self, item, count, parent):
        self.item = item           # Item value
        self.count = count         # Support count of the itemset
        self.parent = parent       # Parent node
        self.children = {}         # Children nodes (item: Node)
        self.link = None 

class FPGrowth:
    def __init__(self, data, minsup):
        self.data = data
        self.root = Node(None,0,None)

    
    def find_frequent_items(self,data, minsup):
        header_table = {}
        for _, item_ls in data.items():
            for item in item_ls:
                header_table[item] = header_table.get(item, 0) + 1
        
        #Sort the dictionary
        header_table = {k: v for k, v in sorted(header_table.items(), key=lambda item: -item[1])}
        print(f"{header_table = }")
        header_table = {k:-1 for k,v in header_table.items() if v>minsup}
        self.l = [*header_table.keys()]
        return header_table 
    
    #Constructing an FPTree
    def construct_fptree(self, data, header_table):
        for _, transaction in data.items():
            ordered_transaction = [item for item in transaction if item in self.l]
            ordered_transaction.sort(key = lambda x:self.l.index(x))
            current_node = self.root
            print(f"{ordered_transaction = }")
            for item in ordered_transaction:
                if item in current_node.children:
                    #Update the count of the already existing node
                    child_node = current_node.children[item]
                    child_node.count += 1
                else:
                    #Create a new node 
                    child_node = Node(item, 1, current_node)
                    current_node.children[item] = child_node
                    #Update header table
                    if item in header_table: #Why does this exist?
                        if header_table[item] == -1:
                            header_table[item] = child_node
                        else:
                            header_node = header_table[item]
                            while header_node.link is not None:
                                header_node =  header_node.link
                            header_node.link = child_node 
                current_node = child_node 

    #Mining an FPTree
    def mine_frequent_patterns(self, header_table, min_support, prefix=[]):
        global id
        frequent_patterns = []
        # Sort items in header table in descending order of frequency
        sorted_items = [item for item in header_table.keys()]
        sorted_items.sort(key=lambda x: (header_table[x].count, x))
        for item in sorted_items:
            new_prefix = prefix + [item]
            support = header_table[item].count
            frequent_patterns.append((new_prefix, support))

            # Build the conditional pattern base
            conditional_dataset = {}
            node = header_table[item]
            while node is not None:
                count = node.count
                path = []
                current_node = node.parent
                while current_node.parent is not None:
                    path.append(current_node.item)
                    current_node = current_node.parent
                for _ in range(count):
                    conditional_dataset[id] = path
                    id += 1
                node = node.link 
            
            # Recursively mine the conditional FP-tree
            conditional_header_table = self.find_frequent_items(conditional_dataset, minsup)
            print(f"Waiting...")
            # conditional_fp_tree, conditional_header_table = construct_fp_tree(conditional_dataset, min_support)
            # if conditional_header_table:
                # frequent_patterns.extend(mine_frequent_patterns(conditional_header_table, min_support, new_prefix))
  
    
        

minsup = 2
FPGrowth_obj = FPGrowth(data, minsup)
header_table = FPGrowth_obj.find_frequent_items(data,minsup)
FPGrowth_obj.construct_fptree(data, header_table)
FPGrowth_obj.mine_frequent_patterns(header_table, minsup, [])
#For debugging
# traversetree(FPGrowth_obj)
# traverseheader(header_table)

header_table = {'f': 4, 'c': 4, 'a': 3, 'm': 3, 'p': 3, 'b': 3, 'l': 2, 'o': 2, 'd': 1, 'g': 1, 'i': 1, 'h': 1, 'j': 1, 'k': 1, 's': 1, 'e': 1, 'n': 1}
ordered_transaction = ['f', 'c', 'a', 'm', 'p']
ordered_transaction = ['f', 'c', 'a', 'm', 'b']
ordered_transaction = ['f', 'b']
ordered_transaction = ['c', 'p', 'b']
ordered_transaction = ['f', 'c', 'a', 'm', 'p']
header_table = {'c': 2, 'f': 2, 'm': 1, 'a': 1, 'p': 1}


KeyboardInterrupt: 